# Observations on Bubble Detection

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 


In [2]:
def build_grid(img, grid_size = (100,100), line_color =(255, 0, 0) ):

    # Iterate over the rows and columns to draw the lines
    for i in range(0, img.shape[0], grid_size[0]):
        cv2.line(img, (0, i), (img.shape[1], i), line_color, 1)
    for j in range(0, img.shape[1], grid_size[1]):
        cv2.line(img, (j, 0), (j, img.shape[0]), line_color, 1)
    return img
def plot_images(img1, img2):
    img1 = build_grid(img1)
    img2 = build_grid(img2)
    f = plt.figure(figsize = (15,10))
    f.add_subplot(1,2, 1)
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    f.add_subplot(1,2, 2)

    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
   
    plt.show(block=True)

def mask(img, lower_threshold = 150, upper_threshold = 250):
    
    # Create the mask for bright pixels
    mask = cv2.inRange(img, lower_threshold, upper_threshold)
    # Apply the mask to the original image to show only bright pixels
    return cv2.bitwise_and(img, img, mask=mask)

def mask_color(img, lower_bound:list, upper_bound:list):
    lower_bound = np.array(lower_bound, dtype = "uint8") 
    upper_bound = np.array(upper_bound, dtype = "uint8")
    mask = cv2.inRange(img, lower_bound, upper_bound)
    return cv2.bitwise_and(img, img, mask =  mask) 
    

# Observations

<li>The reflection of light on the cylinders is significant and is affecting the masked images</li>
<li>We can paint the rods to dark color so as it doesnt get detected</li>
<li>Bubbles are mostly detected in 100 to 150 range</li>

# Detecting Circles

In [6]:
# # img = cv2.medianBlur(img,5)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img)
# image = img.copy()


Performing edge detection to bold out the circles

In [7]:
def merge_circles(circles):
    x = 0
    y = 0
    r = 0
    for circle in circles:
        x += circle[0]
        y += circle[1]
        r = max(r, circle[2])
    x /= len(circles)
    y /= len(circles)
    return (int(x), int(y), int(r))

In [10]:
import cv2
import numpy as np

def get_circles(image, lower_bound = [141,141,121], upper_bound=[150,150,220]):
    """Function Takes in an image both as an array or as a path, Masks it with Red color and returns the detected circles"""
    if isinstance(image, str):
        image = cv2.imread(image)
     
    def mask_color(img, lower_bound:list, upper_bound:list):
        lower_bound = np.array(lower_bound, dtype = "uint8") 
        upper_bound = np.array(upper_bound, dtype = "uint8")
        mask = cv2.inRange(img, lower_bound, upper_bound)
        return cv2.bitwise_and(img, img, mask =  mask) 
   
    # Convert the image to grayscale
    img = mask_color(image, [121,121,121], [150,150,220])
    # cv2.imshow('masked_image', img)
    # cv2.waitKey(0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray', gray)
    # cv2.waitKey(0)

    # # Apply Gaussian blur to reduce noise
    # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # cv2.imshow('blurred', blurred)
    # cv2.waitKey(0)
    # # Apply Canny edge detection
    edges = cv2.Canny(gray, 200, 300)
    # cv2.imshow('edges', edges)
    # cv2.waitKey(0)

    # Detect circular arcs using Hough Transform
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 11, param1=30, param2=38, minRadius=140, maxRadius=400)



    filtered_circles = []

    # Loop over the circles
    if circles is not None:
        # Convert the (x, y) coordinates and radius of the circles to integers
        # circle_dict = {}
        # for circle in circles:
        #     center = (circle[0], circle[1])
        #     radius = circle[2]

        #     # If the circle center and radius are already in the dictionary, increment their count
        #     if (center, radius) in circle_dict:
        #         circle_dict[(center, radius)] += 1
        #     else:
        #         circle_dict[(center, radius)] = 1
        # Select only the circles that occur frequently
        # circles = list(sorted(circle_dict.items(), key=lambda item: item[1], reverse=True))
        print('_______________Circles_____________')
        print(circles)
        
        circles = np.round( circles[0, :]).astype("int")
        for (x, y, r) in circles:
        # Initialize a flag to indicate whether the circle has been filtered
            filtered = False

            # Loop over the filtered circles
            for (fx, fy, fr) in filtered_circles:
                # Calculate the distance between the centers of the circles
                d = np.sqrt((x - fx)**2 + (y - fy)**2)

                # If the distance is smaller than the sum of their radii, remove the circle with the smaller radius
                if d < r + fr:
                    filtered = True
                    if r < fr:
                        break

            # If the circle was not filtered, add it to the list of filtered circles
            if not filtered:
                filtered_circles.append((x, y, r))
                    
        # for (x, y, r) in circles:
        #     cv2.circle(image, (x, y), r, (0, 255, 0), 2)
        # Initialize a list to store the centers of the circles
        # centers = []

        # # Loop over the circles
        # for (x, y, r) in circles:
        #     centers.append([x, y])

        # # Convert the list of centers to a NumPy array
        # centers = np.array(centers, dtype=np.float32)

        # # Perform KMeans clustering on the centers of the circles
        # criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        # flags = cv2.KMEANS_RANDOM_CENTERS
        # compactness, labels, centers = cv2.kmeans(centers, K=2, bestLabels=None, criteria=criteria, attempts=10, flags=flags)

        # # Initialize a dictionary to store the clustered circles
        # clustered_circles = {}
        # # print(circles)
        # # print(circles.shape)
        # # print(labels)
        # # print(labels.shape)
    

        # # Loop over the labels and circles
        # for label, (x, y, r) in zip(labels, circles):
        #     # If the label is not in the dictionary, add it
        #     # print(label[0])
        #     if label[0] not in clustered_circles.keys():
        #         clustered_circles[label[0]] = []

        #     # Add the circle to the list for the corresponding label
        #     clustered_circles[label[0]].append((x, y, r))
        
        # print("________________Clusters__________")
        # print(clustered_circles)

        # Draw the clustered circles in the output image
        colors = [(0, 255, 255), (255, 0, 255),(255, 0, 0), (255, 255, 0), (0, 0, 255)]
        
        # final_circles = [merge_circles(clustered_circles[key]) for key in clustered_circles.keys()]
        
        # print("___________________Final________________")
        # print(final_circles)
        
        for id, circle in enumerate(filtered_circles):
            color = colors[id % len(colors)]
            x,y,r = circle
            cv2.circle(image, (x, y), r, color, 2)
        return image, filtered_circles
    else:
        print('No circles Detected')
        return image, None

        

# Display the output image
# cv2.imshow("Clustered Circles", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [11]:
circled_image, circles = get_circles('121_200_bgr.jpeg')

cv2.imshow('Final Image', circled_image)
cv2.waitKey(0)
print(circles)


No circles Detected
None


### Testing on a video

In [12]:
cam = cv2.VideoCapture(r'Data\plant.mp4')

while cam.isOpened():
    ret, frame = cam.read()
    if not ret or frame is None:
        break
    circled_image, circles = get_circles(frame)
    cv2.imshow('Final Image', circled_image)
    print(circles)
    if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cam.release()
cv2.destroyAllWindows()
    
    
    

_______________Circles_____________
[[[238.5 265.5 164.4]
  [240.5 276.5 164.4]
  [723.5 296.5 157.1]
  ...
  [721.5 462.5 140.8]
  [766.5 502.5 148.1]
  [274.5 591.5 162.9]]]
[(238, 266, 164), (724, 296, 157), (1250, 268, 150), (392, 592, 186)]
_______________Circles_____________
[[[236.5 278.5 169.5]
  [234.5 266.5 163.7]
  [726.5 302.5 151. ]
  ...
  [609.5 435.5 152.8]
  [953.5 201.5 153.6]
  [691.5 526.5 143.8]]]
[(236, 278, 170), (726, 302, 151)]
_______________Circles_____________
[[[239.5 270.5 160.9]
  [245.5 280.5 167.3]
  [727.5 302.5 150.8]
  ...
  [428.5 517.5 141.9]
  [166.5 418.5 150.2]
  [420.5 591.5 153.6]]]
[(240, 270, 161), (728, 302, 151), (466, 592, 223)]
_______________Circles_____________
[[[239.5 280.5 167.9]
  [242.5 269.5 160.8]
  [730.5 295.5 157.2]
  ...
  [487.5 420.5 146.2]
  [290.5 589.5 162.6]
  [379.5 516.5 140.7]]]
[(240, 280, 168), (730, 296, 157), (1236, 276, 152)]
_______________Circles_____________
[[[242.5 279.5 171.6]
  [247.5 269.5 161.7]
  [235